In [19]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.layouts import column

In [30]:
ROOT = Path().resolve().parent

# load data
enrolments = pd.read_csv(ROOT / "converted-data" / "enrolments.csv")
exams = pd.read_csv(ROOT / "converted-data" / "exams.csv")
students = pd.read_csv(ROOT / "converted-data" / "students.csv")
coincidences = pd.read_csv(ROOT / "converted-data" / "coincidences.csv", index_col=0)
dates = pd.read_csv(ROOT / "converted-data" / "dates.csv")
times_expanded = pd.read_csv(ROOT / "converted-data" / "times_expanded.csv")
rooms = pd.read_csv(ROOT / "converted-data" / "rooms.csv")

In [21]:
# convert exam duration from hh:mm to minutes
def parse_duration(time_str):
    h, m = time_str.split(":")
    return int(h) * 60 + int(m)

exams["duration_minutes"] = exams["duration"].apply(parse_duration)

# exam list and duration dict
exam_list = exams["exam"].unique().tolist()
duration = exams.set_index("exam")["duration_minutes"].to_dict()

# Give each row a unique timeslot index
times_expanded["timeslot"] = range(1, len(times_expanded) + 1)

# Convert hours -> minutes
times_expanded["slot_minutes"] = times_expanded["duration_hours"] * 60

timeslots = times_expanded["timeslot"].tolist()
slot_length = times_expanded.set_index("timeslot")["slot_minutes"].to_dict()

# student -> list of exams
students_exams = enrolments.groupby("student")["exam"].apply(list).to_dict()

exams

,exam,description,duration,dept,duration_minutes
0,AA2016E1,"OPERA STUDIES, I",1:30,GM,90
1,AA3008E1,HOLLYWOOD & THE EUROPEAN CINEMA,3:00,AI,180
2,AAA011E1,INTRODUCTION TO ISLAM,1:30,TH,90
3,AAA013E1,INTRODUCTION TO SOCIAL ANTHROPOLOGY,2:00,TH,120
4,AAA022E1,THE SOVIET POLITICAL SYSTEM,1:30,HI,90
...,...,...,...,...,...
795,W32C10E1,PERFORMANCE: HISTORY AND ANALYSIS,1:30,MU,90
796,W32F10E1,INTRO TO NOTATION AND TRANSCRIPTION,1:30,MU,90
797,W32H10E1,RENAISSANCE STUDIES: SACRED MUSIC,1:30,MU,90
798,W32H14E1,BAROQUE STUDIES: VOCAL & INSTRUMENTAL,1:30,MU,90


In [32]:
# define list of room and capacity of each room
room_list = rooms["room"].tolist()
cap = rooms.set_index("room")["capacity"].to_dict()

# count unique students per exam
exam_counts = (
    enrolments.groupby("exam")["student"]
    .nunique()
    .sort_values(ascending=False)
)

size = exam_counts.to_dict()

Setup:

`students_exams`: Dictionary with keys are students and values are required exams.
`exams` : List of exams
`timeslots` : Timeslots for each exams to take place

## Model 1: Basic feasible exam timetabling model

We define the following sets:

- C: the set of exams (courses), indexed by \(i\).
- T: the set of available timeslots, indexed by \(t\).
- S: the set of students, indexed by \(s\).
- $E_s \subseteq C$: the set of exams taken by student \(s\).

### Decision variables

$$
x_{i,t} =
\begin{cases}
1, & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0, & \text{otherwise}.
\end{cases}
$$

## Constraints

### 1. Each exam must be scheduled exactly once
Every exam is assigned to one and only one timeslot:
$$
\sum_{t \in T} x_{i,t} = 1 \qquad \forall i \in C.
$$


### 2. No student may have two exams in the same timeslot
For every student \(s\) and for every pair of exams $(i, j \in E_s)$ with $(i \neq j)$,
they cannot be assigned to the same timeslot:

$$
x_{i,t} + x_{j,t} \le 1
\qquad \forall s \in S,\; \forall i \neq j \in E_s,\; \forall t \in T.
$$

### 3. Exam duration must not exceed timeslot length
Let $\text{dur}_i$ be the duration of exam $i$,  
and let $\text{len}_t$ be the allowable duration of timeslot $t$.  

If an exam is too long to fit into a timeslot, the assignment is forbidden:

$$
x_{i,t} = 0
\qquad \forall i \in C,\; \forall t \in T \text{ such that } \text{dur}_i > \text{len}_t.
$$

## Objective function

For Model 1, our aim is to obtain a feasible schedule.  
Thus, we may use a dummy objective:

$$
\min 0
$$

or equivalently:

$$
\text{Find any feasible assignment satisfying all constraints.}
$$



In [ ]:
# MODEL 1
model1 = gp.Model("Basic feasible exam timetabling model")

# Decision vars x[e,t]
x = model1.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# 1. One timeslot per exam
for e in exam_list:
    model1.addConstr(gp.quicksum(x[e, t] for t in timeslots) == 1,
                name=f"assign_once_{e}")

# 2. No overlapping exams for each student
for s, exam_list_s in students_exams.items():
    if len(exam_list_s) > 1:
        for t in timeslots:
            for i in range(len(exam_list_s)):
                for j in range(i + 1, len(exam_list_s)):
                    e1 = exam_list_s[i]
                    e2 = exam_list_s[j]
                    model1.addConstr(x[e1, t] + x[e2, t] <= 1,
                                name=f"no_overlap_{s}_{e1}_{e2}_{t}")

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model1.addConstr(x[e, t] == 0,
                        name=f"forbidden_duration_{e}_{t}")

# Objective: Feasible schedule
model1.setObjective(0, GRB.MINIMIZE)

# Solve model
model1.optimize()

print("Model 1: Basic feasible exam timetabling model \n")
for e in exam_list:
    for t in timeslots:
        if x[e, t].X > 0.5:
            row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
            print(f"Exam {e} → Day {row['day']}, Start {row['start_time']}, Duration {row['slot_minutes']} min")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1026148 rows, 12800 columns and 2062996 nonzeros
Model fingerprint: 0xfd394944
Variable types: 0 continuous, 12800 integer (12800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 874084 rows and 596 columns
Presolve time: 0.47s
Presolved: 152064 rows, 12204 columns, 314744 nonzeros
Variable types: 0 continuous, 12204 integer (12204 binary)
Performing another presolve...
Presolve removed 114040 rows and 144 columns
Presolve time: 0.45s

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxation presolved: 38024 rows, 12060 columns, 176927 nonzeros

Concurrent spin time: 0.07s

Solve

In [24]:
# Build a list of scheduled exams
schedule_rows = []

for e in exam_list:
    for t in timeslots:
        if x[e,t].X > 0.5:
            row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
            schedule_rows.append({
                "exam": e,
                "day": row["day"],
                "start_time": row["start_time"],
                "duration_min": row["slot_minutes"],
                "timeslot": t
            })

# Turn into a DataFrame
schedule_df = pd.DataFrame(schedule_rows)
schedule_df.sort_values(["day", "start_time"], inplace=True)


timetable = schedule_df.pivot_table(
    index="day",
    columns="start_time",
    values="exam",
    aggfunc=lambda x: ", ".join(x)  # nếu có nhiều exam cùng timeslot (hiếm)
)

timetable.style.set_properties(**{
    'background-color': '#EFEFFF',
    'font-size': '14px',
    'border-color': 'black',
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#D0D0FF')]}
])

timetable


start_time,13:30,16:30,9:00
day,,,
Fri,"C81J4AE1, C81JIAE1, C82OUAE1, F13X03E1, F321C6...","B12320E1, B33545E1, C12331E1, C81HBAE1, D22227...","AA3008E1, B33541E1, C12332E1, C13573E1, C81HCA..."
Mon,"B32321E1, C12321E1, C1P001E1, C81J2AE2, C81MSA...","B11102E1, B12302E1, B33563E1, C8CXPAE1, F311X1...","AAA011E1, AAB025E1, AAB058E1, B11101E1, B31101..."
Sat,NaN,NaN,"B31107E1, B32327E1, B33549E1, C12328E1, C13562..."
Thu,"AAA024E1, C11101E1, C81J2AE1, C82CUAE1, F12I02...","AAA034E1, C13571E1, C42304E1, C82SUAE1, F311B1...","AA2016E1, AAA013E1, C11105E1, C12327E1, C82MCP..."
Tue,"AAA033E1, C12338E1, C81IBAE1, C81J3AE1, C8COUA...","B33548E1, C12324E1, C41101E1, C82MHAE1, F11O02...","B12301E1, B13103E1, B31103E1, B32325E1, B33544..."
Wed,"F11I02E1, F13X01E1, F311X4E1, F321T6E1, F331Z4...","C12325E1, C81IAAE1, C82HUAE1, F13P03E1, F311T1...","AAA022E1, B12303E1, B31105E1, B32326E1, C12329..."


In [29]:
output_notebook()

# 1. Build schedule_df from Model 1

schedule_rows = []
for e in exam_list:
    for t in timeslots:
        if x[e, t].X > 0.5:
            row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
            schedule_rows.append({
                "exam": e,
                "day": row["day"],
                "start_time": row["start_time"],
                "timeslot": t
            })

schedule_df = pd.DataFrame(schedule_rows)

# 2. Create Y-axis as real time labels

# Define fixed exam slot times
time_order = ["9:00", "13:30", "16:30"]     # Saturday only uses 9:00

# Map time strings to numeric Y values (for plotting)
time_to_y = {t: i+1 for i, t in enumerate(time_order)}  
# -> {"9:00":1, "13:30":2, "16:30":3}

x_coords, y_coords, exam_texts = [], [], []

for _, row in schedule_df.iterrows():
    day = row["day"]
    time = row["start_time"]

    # numeric y coordinate
    y_val = time_to_y[time]

    x_coords.append(day)
    y_coords.append(y_val)
    exam_texts.append(row["exam"])

# 3. Combine exams in same cell

df_vis = pd.DataFrame({
    "day": x_coords,
    "y": y_coords,
    "exam": exam_texts
})

df_grouped = df_vis.groupby(["day","y"])["exam"].apply(lambda x: ", ".join(x)).reset_index()

source = ColumnDataSource(df_grouped)

# 4. Plotting

p = figure(
    x_range=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"],
    y_range=[1, len(time_order)],   # numeric range
    height=600,
    title="Exam Timetable (Model 1)",
    tools="tap"
)

p.rect(
    x="day",
    y="y",
    width=0.9,
    height=0.8,
    source=source,
    fill_color="skyblue",
    line_color="black"
)

# Callback
callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const idx = source.selected.indices[0];
    if(idx !== undefined){
        const day = data['day'][idx];
        const y = data['y'][idx];
        const exams = data['exam'][idx];
        alert('Day: ' + day + '\\nTime slot: ' + y + '\\nExams: ' + exams);
    }
""")

source.selected.js_on_change('indices', callback)

# Y-axis labels -> replace numeric with actual time labels
p.yaxis.ticker = [1,2,3]
p.yaxis.major_label_overrides = {
    1: "9:00",
    2: "13:30",
    3: "16:30"
}

p.xaxis.axis_label = "Day"
p.yaxis.axis_label = "Start Time"

p.xaxis.major_label_orientation = 1.2

show(p)

Loading BokehJS ...

## Model 2: Exam timetabling with room assignment

### Sets
$$
\begin{aligned}
C &: \text{ set of exams}, \\
T &: \text{ set of timeslots}, \\
R &: \text{ set of rooms}, \\
S &: \text{ set of students}, \\
E_s &\subseteq C \text{ exams taken by student } s.
\end{aligned}
$$

### Parameters
$$
\begin{aligned}
\text{dur}_i &: \text{ duration of exam } i, \\
\text{len}_t &: \text{ length of timeslot } t, \\
\text{cap}_r &: \text{ capacity of room } r, \\
\text{size}_i &: \text{ number of students enrolled in exam } i.
\end{aligned}
$$

### Decision variables
$$
x_{i,t} =
\begin{cases}
1 & \text{if exam } i \text{ is assigned to timeslot } t, \\
0 & \text{otherwise}
\end{cases}
\qquad
y_{i,r} =
\begin{cases}
1 & \text{if exam } i \text{ is assigned to room } r, \\
0 & \text{otherwise}
\end{cases}
$$

## Constraints

### 1. Each exam must be assigned exactly one timeslot
$$
\sum_{t \in T} x_{i,t} = 1 \qquad \forall i \in C
$$

### 2. Each exam must be assigned exactly one room
$$
\sum_{r \in R} y_{i,r} = 1 \qquad \forall i \in C
$$

### 3. No student may have overlapping exams
$$
x_{i,t} + x_{j,t} \le 1
\qquad \forall s \in S,\; \forall i \neq j \in E_s,\; \forall t \in T
$$

### 4. Room capacity constraint 
If the room is too small, the assignment is forbidden
$$
y_{i,r} = 0
\quad \forall i \in C,\; \forall r \in R \text{ with } \text{cap}_r < \text{size}_i
$$

### 5. Exam duration must fit in the timeslot
$$
x_{i,t} = 0
\quad \forall i \in C,\; \forall t \in T \text{ with } \text{dur}_i > \text{len}_t
$$

### 6. A room cannot host two exams in the same timeslot
$$
x_{i,t} + x_{j,t} + y_{i,r} + y_{j,r} \le 3
\quad \forall r \in R,\; \forall t \in T,\; \forall i < j
$$

## Objective
$$
\min 0
$$

## Model 2B: Exam timetabling with room merging

### Sets
$$
\begin{aligned}
C &: \text{ set of exams}, \\
T &: \text{ set of timeslots}, \\
R &: \text{ set of rooms}, \\
S &: \text{ set of students}, \\
E_s &\subseteq C \text{ exams taken by student } s.
\end{aligned}
$$

### Parameters
$$
\begin{aligned}
\text{dur}_i &: \text{ duration of exam } i, \\
\text{len}_t &: \text{ length of timeslot } t, \\
\text{cap}_r &: \text{ capacity of room } r, \\
\text{size}_i &: \text{ number of students enrolled in exam } i.
\end{aligned}
$$

### Decision Variables
$$
x_{i,t} =
\begin{cases}
1 & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0 & \text{otherwise}
\end{cases}
\qquad
y_{i,r} =
\begin{cases}
1 & \text{if room } r \text{ is used by exam } i, \\
0 & \text{otherwise.}
\end{cases}
$$

## Constraints 
### 1. Exam assigned exactly one timeslot
$$
\sum_{t \in T} x_{i,t} = 1 \qquad \forall i \in C
$$

### 2. Exams may use multiple rooms
(We REMOVE the constraint requiring exactly one room.)

### 3. Room capacity requirement with merging
$$
\sum_{r \in R} cap_r \, y_{i,r} \ge size_i 
\qquad \forall i \in C
$$

### 4. No room can be used by two exams in the same timeslot
$$
x_{i,t} + x_{j,t} + y_{i,r} + y_{j,r} \le 3
\quad \forall r \in R,\; \forall t \in T,\; \forall i<j
$$

### 5. Rooms in a “together group” may be used simultaneously
If a group $G \subseteq R$ is marked “together”, then:
$$
y_{i,r} = y_{i,r'} \qquad \forall r,r' \in G
$$

### 6. Exam duration constraint
$$
x_{i,t} = 0 \qquad \forall i,t \text{ such that } dur_i > len_t
$$

## Objective
$$
\min 0
$$

In [ ]:
## Model 2B: Exam Timetabling with Room Merging (Corrected Version)

### Sets
$$
\begin{aligned}
C &: \text{ set of exams}, \\
T &: \text{ set of timeslots}, \\
R &: \text{ set of rooms}, \\
S &: \text{ set of students}, \\
E_s &\subseteq C \text{ is the set of exams taken by student } s.
\end{aligned}
$$

### Parameters
$$
\begin{aligned}
dur_i &: \text{ duration of exam } i, \\
len_t &: \text{ length of timeslot } t, \\
cap_r &: \text{ capacity of room } r, \\
size_i &: \text{ number of students enrolled in exam } i.
\end{aligned}
$$

### Decision Variables
$$
x_{i,t} =
\begin{cases}
1 & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0 & \text{otherwise},
\end{cases}
\qquad
y_{i,r} =
\begin{cases}
1 & \text{if exam } i \text{ uses room } r, \\
0 & \text{otherwise}.
\end{cases}
$$

---

## Constraints

### 1. Each exam must be assigned exactly one timeslot
$$
\sum_{t \in T} x_{i,t} = 1
\qquad \forall i \in C.
$$

### 2. (No constraint here: multiple rooms allowed for each exam)

### 3. Room capacity requirement (merging allowed)
$$
\sum_{r \in R} cap_r \, y_{i,r} \ge size_i
\qquad \forall i \in C.
$$

### 4A. Linking room usage and timeslot assignment
An exam can only use a room if it is scheduled in some timeslot:
$$
y_{i,r} \le \sum_{t \in T} x_{i,t}
\qquad \forall i \in C,\; \forall r \in R.
$$

### 4B. No room can host two different exams in the same timeslot
For any pair of exams \(i,j\) and any room \(r\):
$$
y_{i,r} + y_{j,r} \le 2 - (x_{i,t} + x_{j,t})
\qquad \forall r \in R,\; \forall t \in T,\; \forall i < j.
$$

This enforces:
- If both \(i\) and \(j\) are assigned to timeslot \(t\), then  
  \(y_{i,r} + y_{j,r} \le 1\).

---

### 5. Rooms in a together-group must be used jointly
For each together-group \(G \subseteq R\):
$$
y_{i,r} = y_{i,r'}
\qquad \forall i \in C,\; \forall r,r' \in G.
$$

---

### 6. Exam cannot be placed in a timeslot that is too short
$$
x_{i,t} = 0
\qquad \forall i \in C,\; \forall t \in T:\; dur_i > len_t.
$$

---

## Objective
(A hard-constraint feasibility model)
$$
\min 0.
$$


In [36]:
# MODEL 2

model2 = gp.Model("Exam timetabling with room assignment")

# Decision variables
x = model2.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")  # exam <-> timeslot
y = model2.addVars(exam_list, room_list, vtype=GRB.BINARY, name="y")  # exam <-> room

# 1. Each exam assigned exactly one timeslot
for e in exam_list:
    model2.addConstr(gp.quicksum(x[e, t] for t in timeslots) == 1,
                     name=f"timeslot_once_{e}")

# 2. Each exam assigned exactly one room
for e in exam_list:
    model2.addConstr(gp.quicksum(y[e, r] for r in room_list) == 1,
                     name=f"room_once_{e}")

# 3. No student overlap constraint
for s, exams_s in students_exams.items():
    for t in timeslots:
        for i in range(len(exams_s)):
            for j in range(i+1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                model2.addConstr(x[e1, t] + x[e2, t] <= 1,
                                 name=f"no_overlap_{s}_{e1}_{e2}_{t}")

# 4. Room capacity constraint
for e in exam_list:
    for r in room_list:
        if size[e] > cap[r]:    # cannot fit
            model2.addConstr(y[e, r] == 0,
                             name=f"cap_forbidden_{e}_{r}")

# 5. Exam duration must fit slot length
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model2.addConstr(x[e, t] == 0,
                             name=f"duration_forbidden_{e}_{t}")

# 6. A room cannot host 2 exams in the same slot
for r in room_list:
    for t in timeslots:
        for i in range(len(exam_list)):
            for j in range(i+1, len(exam_list)):
                e1 = exam_list[i]
                e2 = exam_list[j]
                
                # If both exams use room r and timeslot t → impossible
                model2.addConstr(x[e1,t] + x[e2,t] + y[e1,r] + y[e2,r] <= 3,
                                 name=f"room_conflict_{e1}_{e2}_{r}_{t}")

# Objective = feasibility
model2.setObjective(0, GRB.MINIMIZE)

model2.optimize()

print("Model 2: Exam timetabling with room assignment \n")
for e in exam_list:
    assigned_t = [t for t in timeslots if x[e,t].X > 0.5][0]
    assigned_r = [r for r in room_list if y[e,r].X > 0.5][0]
    print(f"Exam {e} -> Timeslot {assigned_t}, Room {assigned_r}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 82847667 rows, 25600 columns and 329349315 nonzeros
Model fingerprint: 0xeb3bd66a
Variable types: 0 continuous, 25600 integer (25600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 0 rows and 0 columns (presolve time = 301s)...
Presolve removed 0 rows and 0 columns (presolve time = 319s)...
Presolve removed 0 rows and 0 columns (presolve time = 324s)...
Presolve removed 0 rows and 0 columns (presolve time = 328s)...
Presolve removed 0 rows and 0 columns (presolve time = 343s)...
Presolve removed 486659 rows and 3619 columns (presolve time = 374s)...
Presolve removed 486659 rows and 3619 columns
Presolve time: 435.34s

Explored 0 nodes (0 simplex itera

AttributeError: Unable to retrieve attribute 'X'

## Model 4: Minimizing same day student conflicts

### Additional Set
$$
D: \text{ set of days}
$$

Each timeslot $t \in T$ belongs to exactly one day denoted $\text{day}(t) \in D$.

### New decision variable
$$
c_{s,i,j} =
\begin{cases}
1 & \text{if student } s \text{ has exams } i \text{ and } j \text{ on the same day}, \\
0 & \text{otherwise}.
\end{cases}
$$

(Defined only for pairs $i < j$ where student $s$ is enrolled in both.)

## Constraints

### 7. Define same-day conflict indicator
For all students $s$, for all exam pairs $(i < j \in E_s)$:

$$
c_{s,i,j} \ge x_{i,t} + x_{j,t'} - 1
\quad \text{for all } t, t' \in T \text{ such that } \text{day}(t) = \text{day}(t').
$$

(This forces $(c_{s,i,j} = 1)$ if the two exams are scheduled on the same day.)

## Objective: Minimize total same-day conflicts

$$
\min \sum_{s \in S} \sum_{i < j \in E_s} c_{s,i,j}
$$

### Notes
- Model 3 **keeps all constraints from Model 1 and Model 2** (feasibility).  
- Model 3 only **adds conflict indicators and minimization objective**.

In [ ]:
timeslot_day = times_expanded.set_index("timeslot")["day"].to_dict()

# MODEL 4

model4 = gp.Model("Minimizing same day student conflicts")

# Decision variables
x = model4.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")
y = model3.addVars(exam_list, room_list, vtype=GRB.BINARY, name="y")

# Timeslot -> Day
timeslot_day = times_expanded.set_index("timeslot")["day"].to_dict()

# Build conflict pairs per student
conflict_pairs = []
for s, exs in students_exams.items():
    for i in range(len(exs)):
        for j in range(i+1, len(exs)):
            conflict_pairs.append((s, exs[i], exs[j]))

# Conflict indicator
c = model3.addVars(conflict_pairs, vtype=GRB.BINARY, name="c")

# 1. Each exam assigned exactly once
for e in exam_list:
    model3.addConstr(gp.quicksum(x[e,t] for t in timeslots) == 1)

# 2. Each exam assigned to exactly one room
for e in exam_list:
    model3.addConstr(gp.quicksum(y[e,r] for r in room_list) == 1)

# 3. No student overlap
for s, exams_s in students_exams.items():
    for t in timeslots:
        for i in range(len(exams_s)):
            for j in range(i+1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                model3.addConstr(x[e1,t] + x[e2,t] <= 1)

# 4. Room capacity constraint
for e in exam_list:
    for r in room_list:
        if size[e] > cap[r]:
            model3.addConstr(y[e,r] == 0)

# 5. Exam duration must fit
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model3.addConstr(x[e,t] == 0)

# 6. Room cannot host 2 exams in same timeslot
for r in room_list:
    for t in timeslots:
        for i in range(len(exam_list)):
            for j in range(i+1, len(exam_list)):
                e1 = exam_list[i]
                e2 = exam_list[j]
                model3.addConstr(x[e1,t] + x[e2,t] + y[e1,r] + y[e2,r] <= 3)

# 7. Same-day conflicts for Model 3
for (s, e1, e2) in conflict_pairs:
    for t1 in timeslots:
        for t2 in timeslots:
            if timeslot_day[t1] == timeslot_day[t2]:
                model3.addConstr(
                    c[s,e1,e2] >= x[e1,t1] + x[e2,t2] - 1
                )

# Objective: Minimize same-day conflicts
model3.setObjective(
    gp.quicksum(c[s,e1,e2] for (s,e1,e2) in conflict_pairs),
    GRB.MINIMIZE
)

model3.optimize()

print("Model 3: Minimizing same day student conflicts \n")
for e in exam_list:
    assigned_t = [t for t in timeslots if x[e,t].X > 0.5][0]
    assigned_r = [r for r in room_list if y[e,r].X > 0.5][0]
    print(f"Exam {e}: Timeslot {assigned_t} ({timeslot_day[assigned_t]}) — Room {assigned_r}")

## Model 5: Minimizing consecutive or overnight exam conflicts

### Additional Definition
Each timeslot $t \in T$ has an associated ordering index $\text{ord}(t)$ such that \( \text{ord}(t+1) \) is the next available timeslot.

A pair of exams is considered consecutive for a student if they occur in timeslots $t$ and $t'$ with $\text{ord}(t') = \text{ord}(t) + 1$.

### New decision variable
$$
o_{s,i,j} =
\begin{cases}
1 & \text{if student } s \text{ has exams } i \text{ and } j \text{ in consecutive timeslots}, \\
0 & \text{otherwise}.
\end{cases}
$$

# New constraints for model 4

### 8. Consecutive timeslot conflict indicator
For all students $s$, exams $i < j \in E_s$, and all consecutive slots $t, t'$:

$$
o_{s,i,j} \ge x_{i,t} + x_{j,t'} - 1
\qquad \forall s,\, \forall i<j \in E_s,\, \forall (t,t') \text{ such that } \text{ord}(t') = \text{ord}(t)+1.
$$

Symmetrically, the reverse order is also a conflict:

$$
o_{s,i,j} \ge x_{i,t'} + x_{j,t} - 1
\qquad \forall s,\, \forall i<j \in E_s,\, \forall (t,t') \text{ consecutive}.
$$

## Objective for model 4
Minimize the number of consecutive or overnight exam conflicts:

$$
\min \sum_{s \in S} \sum_{i<j \in E_s} o_{s,i,j}
$$

## Model 6: Institutional constraints and priority scheduling


Model 6 extends the previous models by incorporating institutional, departmental, and logistical rules extracted from the University of Nottingham dataset. No new decision variables are added. All constraints below apply to the existing variables:


$$
x_{i,t} =
\begin{cases}
1 & \text{if exam $i$ is scheduled in timeslot $t$,} \\
0 & \text{otherwise,}
\end{cases}
\qquad
y_{i,r} =
\begin{cases}
1 & \text{if exam $i$ is assigned room $r$,} \\
0 & \text{otherwise.}
\end{cases}
$$


Let:
$A_i \subseteq T$ be allowed timeslots for exam $i$.


$R_i \subseteq R$ be allowed rooms for exam $i$.


$P_i$ = priority weight for earliness.


$\mathrm{ord}(t)$ be chronological index of timeslot $t$.


$\mathcal{G}$ be set of standard coincidence groups.


$\mathcal{G}^{\mathrm{seq}}$ be special coincidence groups requiring ordered execution within a single long timeslot.


$\mathcal{G}^{\mathrm{one}}$ be coincidence groups required to share exactly one room and exclude all other exams.


$\mathcal{S}$ be set of exam pairs $(i,j)$ where $i$ must occur before $j$.


$\mathcal{F}$ be set of forbidden room–timeslot pairs $(r,t)$.


$\mathcal{D}^{\mathrm{AM}},\ \mathcal{D}^{\mathrm{PM}}$ be AM-only or PM-only exam sets.


$\mathcal{X}$ be set of spread-out exam groups.


$\mathcal{R}^{\mathrm{comb}}_i$ be pairs of rooms $\,(r,r')$ that must be simultaneously assigned to exam $i$.


## Constraint


### 1. Allowed timeslot constraints


$$
x_{i,t} = 0 \qquad \forall i \in C,\ \forall t \notin A_i.
$$


This encodes restrictions such as:


$K1AHWAE2$: allowed only in AM slots.


$V13101E1$: allowed only in Thursday PM slots.


Exams restricted to the calendar window Jan 23--24.




### 2. Allowed room constraints


$$
y_{i,r} = 0 \qquad \forall i \in C,\ \forall r \notin R_i.
$$


Examples encoded in $R_i$ include:


ART exams restricted to **ART-LECTURE** or **ART-SEMINAR** rooms.


Chemistry exams requiring **CHEMISTRY** laboratories.


Architecture exams requiring **ARCHTCT-LR1**.


$AA3008E1$ requiring the special two-phase assignment (see sequencing constraints below).


### 3. Standard coincidence constraints


For every coincidence group $G \in \mathcal{G}$,all exams must occur in the same timeslot:
$$
x_{i,t} = x_{j,t}
\qquad \forall G\in\mathcal{G},\ \forall i,j \in G,\ \forall t\in T.
$$


These include groups such as:
$$
\{ \text{LK44FAE1, LK44GAE1, LK44SAE1} \},
\quad
\{ \text{M11341E1, M11345E1} \},
\quad \text{etc.}
$$


### 4. Special ``one-after-the-other'' coincidence groups


For any group $G \in \mathcal{G}^{\mathrm{seq}}$ 
(e.g.\ $\{\text{C13563E1}, \text{C13571E1}, \text{C13572E1}\}$),
all exams must share the same timeslot:
$$
x_{i,t} = x_{j,t}
\qquad \forall G\in\mathcal{G}^{\mathrm{seq}},\ \forall i,j\in G,\ \forall t.
$$


Additionally, exams that must occur in order (back-to-back within the same slot)
satisfy:
$$
\mathrm{orderWithinSlot}(i) < \mathrm{orderWithinSlot}(j)
\qquad \forall (i,j) \in \mathcal{G}^{\mathrm{seq}}_{\mathrm{ordered}}.
$$


(If the model treats timeslots as atomic, this may be relaxed.)


### 5. One-room-only coincidence groups


For groups $G \in \mathcal{G}^{\mathrm{one}}$
(e.g.\ $\{\text{H22C20E1}, \text{H23C20E1}, \text{H24C20E1}, \text{H23CEOE1}\}$):


$$
x_{i,t} = x_{j,t}
\qquad \forall i,j\in G,\ \forall t.
$$


All exams must share exactly one room:
$$
y_{i,r} = y_{j,r}
\qquad \forall i,j\in G,\ \forall r\in R.
$$


No other exam may use that room:
$$
\sum_{k \notin G} y_{k,r} = 0
\qquad \forall r \text{ with } y_{i,r} = 1.
$$


### 6. Sequencing constraints (before/after)


For each ordered pair $(i,j) \in \mathcal{S}$:
$$
\sum_{t\in T} \mathrm{ord}(t)\, x_{i,t}
<
\sum_{t\in T} \mathrm{ord}(t)\, x_{j,t}.
$$


Examples include:
$$
\text{F13P03E1} \prec \text{F13X03E1},
\qquad
\text{F13P05E1} \prec \text{F13X04E1},
\qquad
\text{H3BFM2E1} \text{ immediately before } \text{H3BFM2E2}.
$$


### 7. Room unavailability and forbidden pairs


For each forbidden room--timeslot pair $(r,t)\in\mathcal{F}$,
such as TRENT-B46 on Friday AM:
$$
y_{i,r} + x_{i,t} \le 1
\qquad \forall i\in C.
$$


\subsubsection*{8. AM-only and PM-only exam constraints}


$$
x_{i,t} = 0
\qquad \forall i \in \mathcal{D}^{\mathrm{AM}},\ \forall t \notin \mathrm{AM}.
$$


$$
x_{i,t} = 0
\qquad \forall i \in \mathcal{D}^{\mathrm{PM}},\ \forall t \notin \mathrm{PM}.
$$


### 9. Spread-out constraints


For each spread-out group $X\in\mathcal{X}$,
exams must not occur too close together. For every $i,j\in X$:
$$
\lvert\, \mathrm{ord}(t_i) - \mathrm{ord}(t_j) \,\rvert \ge \Delta_X,
$$
where
$$
t_i = \sum_{t\in T} \mathrm{ord}(t)\, x_{i,t}.
$$


Groups include:
$$
\{\text{Q33211E1},\ \text{Q33307E1},\ \text{Q33308E1}\},
\qquad
\{\text{B12301E1},\text{B12302E1},\text{B12303E1},\text{B12320E1}\},
\quad \text{etc.}
$$


### 10. Combined-room requirements


If exam $i$ must use both rooms $r$ and $r'$ simultaneously:
$$
y_{i,r} = y_{i,r'}
\qquad \forall (r,r') \in \mathcal{R}^{\mathrm{comb}}_i.
$$


This includes exams requiring POPE-A13 and POPE-A14 jointly.


### 11. Earliness penalty


Define:
$$
\mathrm{penalty}_i
=
\sum_{t\in T} P_i \cdot \mathrm{ord}(t) \cdot x_{i,t}.
$$


### 12.  Model 5 Objective


$$
\min
\left(
\alpha \sum_{s,i,j} c_{s,i,j}
\;+\;
\beta \sum_{s,i,j} o_{s,i,j}
\;+\;
\gamma \sum_{i\in C} \mathrm{penalty}_i
\right).
$$


Where:


$c_{s,i,j}$ = consecutive-conflict indicator,


$o_{s,i,j}$ = same-day conflict indicator,


$\alpha,\beta,\gamma$ = user-defined weights.